In [ ]:
# Импорт необходимых библиотек
import numpy as np
import pandas as pd
import scrapy 
from scrapy.crawler import CrawlerProcess
import bs4
import requests

In [ ]:
class Spyder(scrapy.Spider):
    name = 'Spyder'

    # Страницы с отзывами на все категории, связанные с мобильными телефонами
    def start_requests(self):
        urls_1 = ['https://otziv-otziv.ru/katalog/smartphones/?page=' + str(n) for n in range(1, 81)]
        urls_2 = ['https://otziv-otziv.ru/katalog/phones/?page=' + str(n) for n in range(1, 26)]
        urls_3 = ['https://otziv-otziv.ru/katalog/mobilnye-telefony/?page=' + str(n) for n in range(1, 709)]
        urls = urls_1 + urls_2 + urls_3
        for url in urls:
            yield scrapy.Request(url = url, callback = self.parse_page)
    
    # Парсинг ссылок на модели
    def parse_page(self, response):
        model_links = response.css('div.content > a::attr(href)').extract()
        model_links = list(map(lambda x: 'https://otziv-otziv.ru/' + x, model_links))
        for model_link in model_links:
            yield response.follow(url = model_link, callback = self.parse_model)
    
    # Парсинг страниц с отзывами для каждой модели
    def parse_model(self, response):
        last_child = int(response.css('ul.pagination > li:nth-last-of-type(3) > a::text').extract()[0])
        review_page_links = [response.url + '?page=' + str(n) for n in range(1, last_child + 1)]
        for review_page_link in review_page_links:
            yield response.follow(url = review_page_link, callback = self.parse_comments)
            
    # Парсинг самих отзывов
    def parse_comments(self, response):
        try:
            # Блоки div с комментариями
            req = requests.get(response.url)
            parser = bs4.BeautifulSoup(req.text, 'lxml')
            div_blocks = parser.findAll('div', attrs={'class':'container-reviews collapsible collapsed'})
            
            # Парсинг блоков, где есть комментарий
            for info in div_blocks:
                comment_ind = info.text.find('Комментарий:')
                if comment_ind == -1:
                    continue
                else:
                    comment = info.text[comment_ind+12:].replace('\n', ' ').strip()
                    rating = int(info.find('div', attrs={'class':'stars-container'}).attrs['title'][-1])
                    reviews.append((comment, rating))
                    
        except requests.exceptions.ConnectionError:
            reviews.append(('ошибка подключения', -1))

In [ ]:
# Запуск краулера !!!Осторожно, занял 6.5 часов!!!
reviews = []
process = CrawlerProcess()
process.crawl(Spyder)
process.start()

In [ ]:
# Сохранение в csv-файл
data = pd.DataFrame(reviews, columns = ['Review', 'Rating']).to_csv('data.csv', index = False)